In [ ]:
# Load libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import string
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib
from matplotlib import pyplot as plt
import numpy as np

# **1- Importing the data :**

Commence par charger tes données et afficher les premières lignes

In [ ]:
from google.colab import files
import io

# Charger le fichier
uploaded = files.upload()

Saving Base_Commentaire (6).xlsx to Base_Commentaire (6).xlsx


In [ ]:
#Loading data
data=pd.read_excel('Base_Commentaire.xlsx')

In [ ]:
# Affichage des données
data.head()

,User_Name,Commentaire,Created_time
0,Mona Jamal,من هو الشخص الذي يقوم بزيــــارة بروفايـــلك ي...,"19 févr., à 22:55"
1,Hafedh Bouhani,👎👎👎,"19 févr., à 19:36"
2,Mansouri Afef,بربي يمولو المشاريع الصغري???,"19 févr., à 20:27"
3,Sam Hd,علاه الي يعيش في تونس متعطوشو نفس حظوظ الي يعي...,"19 févr., à 19:30"
4,Mohamed Mohamed,رجعولي فلوسي لا تباركلكم قصيتولي 90دينار جانفي...,"19 févr., à 19:18"


# **2- Exploring the data :**
Examine les informations générales sur le dataset pour comprendre sa structure et les types de données qu'il contient.

In [ ]:
# Afficher des informations générales sur le dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   User_Name     1141 non-null   object
 1   Commentaire   1106 non-null   object
 2   Created_time  1139 non-null   object
dtypes: object(3)
memory usage: 27.6+ KB


--> Le DataFrame contient un total de 1173 lignes réparties sur 3 colonnes : 'User_Name', 'Commentaire' et 'Created_time'.
Les colonnes 'User_Name', 'Commentaire' et 'Created_time' contiennent respectivement 1141, 1106 et 1139 valeurs non-nulles,
ce qui signifie qu'il y a quelques valeurs manquantes dans chacune des colonnes.
Toutes les colonnes sont de type 'object', ce qui indique qu'elles contiennent des chaînes de caractères.
L'utilisation de la mémoire pour ce DataFrame est d'environ 27,6 KB.

# **3- Data preparation**
- Gestion des Données Manquantes

Identifie les valeurs manquantes et décide comment les gérer.

In [ ]:
#copie de data
df = data.copy()

In [ ]:
#Supprimer les colonnes
df = df.drop(['User_Name'], axis=1)
df = df.drop(['Created_time'], axis=1)
df

,Commentaire
0,من هو الشخص الذي يقوم بزيــــارة بروفايـــلك ي...
1,👎👎👎
2,بربي يمولو المشاريع الصغري???
3,علاه الي يعيش في تونس متعطوشو نفس حظوظ الي يعي...
4,رجعولي فلوسي لا تباركلكم قصيتولي 90دينار جانفي...
...,...
1168,"Bravo la STB,,,,,,,,,,,,"
1169,bravo ya3tikom saha
1170,Bravo S T B excellente initiative
1171,Bravo


--> Pour créer un modèle IA destiné à un chatbot, nous n'avons besoin que du contenu des commentaires.
Les colonnes 'User_Name' et 'Created_time' ne sont pas nécessaires
car elles ne contribuent pas à la compréhension ou à la génération des réponses du chatbot.
En supprimant ces colonnes et nous nous concentrons uniquement sur les informations pertinentes, ce qui peut améliorer les performances du modèle.

In [ ]:
#nombre des missing values
df.isnull().mean()*100

Commentaire    5.71185
dtype: float64

In [ ]:
# Supprimer des missing values
df = df.dropna()

--> Le DataSet contient 6 % de valeurs manquantes. Ensuite, nous avons supprimé les lignes contenant des valeurs manquantes pour éviter les biais et améliorer la performance du modèle. Cela assure que notre modèle repose sur un jeu de données propre et complet, offrant des réponses plus précises et cohérentes.


In [ ]:
# Verification des missing values
df.isnull().mean()*100

Commentaire    0.0
dtype: float64


*   Nettoyage des Données

Effectue des opérations de nettoyage, notamment pour les données textuelles.

In [ ]:
#Supprimer les caractères spéciaux tel que ' !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '

def remove_b(txt):
    text_no="".join([c for c in txt if c not in string.punctuation])
    return text_no
df['Commentaire']= df['Commentaire'].astype(str).apply(lambda x: remove_b (x) )
df

<ipython-input-12-f5c36cf06f67>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire']= df['Commentaire'].astype(str).apply(lambda x: remove_b (x) )


,Commentaire
0,من هو الشخص الذي يقوم بزيــــارة بروفايـــلك ي...
1,👎👎👎
2,بربي يمولو المشاريع الصغري
3,علاه الي يعيش في تونس متعطوشو نفس حظوظ الي يعي...
4,رجعولي فلوسي لا تباركلكم قصيتولي 90دينار جانفي...
...,...
1168,Bravo la STB
1169,bravo ya3tikom saha
1170,Bravo S T B excellente initiative
1171,Bravo


In [ ]:
!pip install emoji==1.7.0 # Install older version of emoji that contains the necessary function

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=3628ef39add44c7cb9db85e59777cf04436689a8ccfff29713a37f797a122393
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [ ]:
#supprimer les emojis
import emoji
def strip_emoji(text):

    new_text=re.sub(emoji.get_emoji_regexp(),r"",text)
    return new_text

df['Commentaire'] = df['Commentaire'].astype(str).apply(lambda x: strip_emoji(x))

df

<ipython-input-14-742d988eeed8>:5: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  new_text=re.sub(emoji.get_emoji_regexp(),r"",text)
<ipython-input-14-742d988eeed8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire'] = df['Commentaire'].astype(str).apply(lambda x: strip_emoji(x))


,Commentaire
0,من هو الشخص الذي يقوم بزيــــارة بروفايـــلك ي...
1,
2,بربي يمولو المشاريع الصغري
3,علاه الي يعيش في تونس متعطوشو نفس حظوظ الي يعي...
4,رجعولي فلوسي لا تباركلكم قصيتولي 90دينار جانفي...
...,...
1168,Bravo la STB
1169,bravo ya3tikom saha
1170,Bravo S T B excellente initiative
1171,Bravo


In [ ]:
# Vérifier si ilya les lignes vides après avoir supprimé les emojis et les caractéres speciaux
empty_rows = df[df['Commentaire'].str.strip() == '']
empty_rows.count()

Commentaire    8
dtype: int64

In [ ]:
# Supprimer les lignes où n'importe quelle des colonnes textuelles est vide après avoir supprimé les espaces
df = df[(df['Commentaire'].str.strip() != '')]

# Réinitialiser l'index du DataFrame nettoyé
df.reset_index(drop=True, inplace=True)

# Afficher les premières lignes après nettoyage
df.head()

,Commentaire
0,من هو الشخص الذي يقوم بزيــــارة بروفايـــلك ي...
1,بربي يمولو المشاريع الصغري
2,علاه الي يعيش في تونس متعطوشو نفس حظوظ الي يعي...
3,رجعولي فلوسي لا تباركلكم قصيتولي 90دينار جانفي...
4,Slm belehi doubel salaire mazel wela


In [ ]:
# Vérifier les lignes vides après avoir supprimé les espaces
empty_rows = df[df['Commentaire'].str.strip() == '']
empty_rows.count()

Commentaire    0
dtype: int64

--> Après la suppression des emojis et des caractères spéciaux, je vois que certaines lignes contiennent des chaînes vides, ce qui n'apporte aucune valeur informative au modèle. Pour garantir que notre ensemble de données est utile et pertinent, nous avons décidé de supprimer ces lignes vides.

In [ ]:
#Normalisation des Textes Convertit le texte en minuscules avec text.lower().
df['Commentaire']=df['Commentaire'].str.lower()
df

<ipython-input-18-bf5d58ecf1d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire']=df['Commentaire'].str.lower()


,Commentaire
0,من هو الشخص الذي يقوم بزيــــارة بروفايـــلك ي...
1,بربي يمولو المشاريع الصغري
2,علاه الي يعيش في تونس متعطوشو نفس حظوظ الي يعي...
3,رجعولي فلوسي لا تباركلكم قصيتولي 90دينار جانفي...
4,slm belehi doubel salaire mazel wela
...,...
1093,bravo la stb
1094,bravo ya3tikom saha
1095,bravo s t b excellente initiative
1096,bravo


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#tokenized Utilise word_tokenize de NLTK pour découper le texte en mots.

df['Commentaire'] = df['Commentaire'].apply(nltk.word_tokenize)
df

<ipython-input-20-2cac2749cc00>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire'] = df['Commentaire'].apply(nltk.word_tokenize)


,Commentaire
0,"[من, هو, الشخص, الذي, يقوم, بزيــــارة, بروفاي..."
1,"[بربي, يمولو, المشاريع, الصغري]"
2,"[علاه, الي, يعيش, في, تونس, متعطوشو, نفس, حظوظ..."
3,"[رجعولي, فلوسي, لا, تباركلكم, قصيتولي, 90دينار..."
4,"[slm, belehi, doubel, salaire, mazel, wela]"
...,...
1093,"[bravo, la, stb]"
1094,"[bravo, ya3tikom, saha]"
1095,"[bravo, s, t, b, excellente, initiative]"
1096,[bravo]


In [ ]:
#stemming Utilise le PorterStemmer de NLTK pour réduire les mots à leur racine.
ps=PorterStemmer()
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

df['Commentaire'] = df['Commentaire'].apply(lambda x: stemming(x))
df

<ipython-input-21-6f433badf679>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire'] = df['Commentaire'].apply(lambda x: stemming(x))


,Commentaire
0,"[من, هو, الشخص, الذي, يقوم, بزيــــارة, بروفاي..."
1,"[بربي, يمولو, المشاريع, الصغري]"
2,"[علاه, الي, يعيش, في, تونس, متعطوشو, نفس, حظوظ..."
3,"[رجعولي, فلوسي, لا, تباركلكم, قصيتولي, 90دينار..."
4,"[slm, belehi, doubel, salair, mazel, wela]"
...,...
1093,"[bravo, la, stb]"
1094,"[bravo, ya3tikom, saha]"
1095,"[bravo, s, t, b, excellent, initi]"
1096,[bravo]


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#lemmatization
wn=nltk.WordNetLemmatizer()
def lemmatization(token_text):
    text = [wn.lemmatize(word) for word in token_text]
    return text

df['Commentaire'] = df['Commentaire'].apply(lambda x: lemmatization(x))
df

<ipython-input-23-d4fb060533d2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire'] = df['Commentaire'].apply(lambda x: lemmatization(x))


,Commentaire
0,"[من, هو, الشخص, الذي, يقوم, بزيــــارة, بروفاي..."
1,"[بربي, يمولو, المشاريع, الصغري]"
2,"[علاه, الي, يعيش, في, تونس, متعطوشو, نفس, حظوظ..."
3,"[رجعولي, فلوسي, لا, تباركلكم, قصيتولي, 90دينار..."
4,"[slm, belehi, doubel, salair, mazel, wela]"
...,...
1093,"[bravo, la, stb]"
1094,"[bravo, ya3tikom, saha]"
1095,"[bravo, s, t, b, excellent, initi]"
1096,[bravo]


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Afficher les mots les plus fréquents :**

In [ ]:
from collections import Counter # imports the Counter object from the collections module

# Combiner toutes les listes de mots en une seule liste
all_words = [word for comment in df['Commentaire'] for word in comment]

# Compter la fréquence des mots
word_freq = Counter(all_words)

# Convertir le compteur en DataFrame pour une visualisation facile
word_freq_df = pd.DataFrame(word_freq.items(), columns=['Mot', 'Fréquence'])

# Trier le DataFrame par fréquence décroissante
word_freq_df = word_freq_df.sort_values(by='Fréquence', ascending=False).reset_index(drop=True)

# Afficher le DataFrame
word_freq_df

,Mot,Fréquence
0,و,277
1,في,265
2,stb,227
3,فرع,182
4,من,179
...,...,...
5425,انني,1
5426,ماشي,1
5427,magnifiqu,1
5428,laffich,1


In [ ]:
# Dictionnaire perso
stopwords_tunisien = set(["belehi","voilà","کان","ad","addo","adh","تباركلكم","علاه","بربي","كيفاش","شاء","وماهو","وان","ان","و","بهذا","هذا","الي","كان","بلي","طكططططط","jean","jaur","f","tt","k","lel","انا","وهذ","الا","وهات",
                  "cett","mt","وكل","لكل","او","الكل","084","1","الا","la","le","ووووووو","وا","باش","jai","ay","w","ra","20fev2021d","16022021",
                  "10ما","074","هههههه","bb","ok","نحب","عندي","b","10","03","063","068","10000metr","10آلاف","14","140","15","15000"
                  ,"1400","149","000","10000000","1200","1250","1300","1345","1350","083","087","12","11900","115","11",
                  "1700d","100","084رجلا",
                  "15d","17","15سنة",
                  "1840","19","1977","يوم14","1984","1981الى","an18"
                 "10000metr","يوم14","1800قداش",
                 "18an","1993","1998","1999","1an","2","20","200","2007","2008",
                 "10000metr","les","26"])

In [ ]:

stopwords_arabic= nltk.corpus.stopwords.words('arabic')
stopwords_french= nltk.corpus.stopwords.words('french')
stopwords_english= nltk.corpus.stopwords.words('english')

stopwords = set(stopwords_arabic)|set(stopwords_french)|set(stopwords_english)|set(stopwords_tunisien)

#Supprimer stopwords

def remove_stopwords(txt):
    txt_clean = [word for word in txt if word not in stopwords]
    return txt_clean

df['Commentaire'] = df['Commentaire'].apply(lambda x: remove_stopwords(x))

df

<ipython-input-28-6cd79404f414>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire'] = df['Commentaire'].apply(lambda x: remove_stopwords(x))


,Commentaire
0,"[الشخص, يقوم, بزيــــارة, بروفايـــلك, يوميا, ..."
1,"[يمولو, المشاريع, الصغري]"
2,"[يعيش, تونس, متعطوشو, حظوظ, يعيش, البرا؟]"
3,"[رجعولي, فلوسي, قصيتولي, 90دينار, وفيفري]"
4,"[slm, doubel, salair, mazel, wela]"
...,...
1093,"[bravo, stb]"
1094,"[bravo, ya3tikom, saha]"
1095,"[bravo, excellent, initi]"
1096,[bravo]


In [ ]:
#verifier si ilya des espace ou non
def esp (text):
    if ' ' in text :
        return "True"
s = df['Commentaire'].astype(str).apply(lambda x: esp(x))
s.count()

1027

In [ ]:
#supprimer les espace
def supprimer_espace(txt):
    tokens = re.sub(r"\s+","",txt)#
    return tokens

df['Commentaire'] = df['Commentaire'].astype(str).apply(lambda x: supprimer_espace(x))
df

<ipython-input-30-665d2dc4b691>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Commentaire'] = df['Commentaire'].astype(str).apply(lambda x: supprimer_espace(x))


,Commentaire
0,"['الشخص','يقوم','بزيــــارة','بروفايـــلك','يو..."
1,"['يمولو','المشاريع','الصغري']"
2,"['يعيش','تونس','متعطوشو','حظوظ','يعيش','البرا؟']"
3,"['رجعولي','فلوسي','قصيتولي','90دينار','وفيفري']"
4,"['slm','doubel','salair','mazel','wela']"
...,...
1093,"['bravo','stb']"
1094,"['bravo','ya3tikom','saha']"
1095,"['bravo','excellent','initi']"
1096,['bravo']


--> Puisque les commentaires ecrire par des tunisien donc j'ai identifier les mots les plus fréquents dans les commentaires.
et j'ai sélectionner manuellement les mots qui sont des stopwords dans le dialecte tunisien et les enregistrez dans un fichier texte pour
utiliser a nettoyer les données textuelles.

In [ ]:
#df.to_excel(r"C:\Users\LENOVO\Desktop\Projet_PFE\Commentaire_nettoyer.xlsx")
df.to_excel('Commentaire_nettoyer.xlsx', index=False)